In [2]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime
city_popu=pd.read_hdf('./Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])


import os
dfo=pd.DataFrame()

for f in os.listdir('./DataSource/Haidilao/')[-1:]:
    # if any(i in f for i in ['2021-12']):
    print(f)
    date=f.split('_')[-1].split('.')[0]
    d=pd.read_csv('./DataSource/Haidilao\%s' % f)
    dfo=pd.concat([dfo,d],axis=0)

old=pd.read_hdf(r'Processed_Data/haidilao.h5',key='1229')
new=pd.concat([old,dfo])
new=new.drop_duplicates()
new.to_hdf(r'Processed_Data/haidilao.h5',key='1229')

hdl.queue.2023-01-03.csv


In [3]:
df=new.copy()
df['WeekDay']=pd.to_datetime(df['排队日期']).dt.strftime('%A')
df['城市']=df['城市'].str.replace('市',"")
df=df.rename(columns={'排队日期':'dt'})

def calculate_paidui(x):
    small=0
    mid=0
    big=0

    if x['小桌排号']==0 or x['小桌叫号']==0:
        small=0
    else:
        small=(x['小桌排号']-x['小桌叫号'])*2
        if small<0:
            small=0

    if x['中桌排号']==0 or x['中桌叫号']==0:
        mid=0
    else:
        mid=(x['中桌排号']-x['中桌叫号'])*4
        if mid<0:
            mid=0
    
    if x['大桌排号']==0 or x['大桌叫号']==0:
        big=0
    else:
        big=(x['大桌排号']-x['大桌叫号'])*4
        if big<0:
            big=0
    
    return small+mid+big

# df['排队人数']=+(df['中桌排号']-df['中桌叫号'])*4+(df['大桌排号']-df['大桌叫号'])*4

df['排队人数']=df.apply(lambda x:calculate_paidui(x),axis=1)

c1=df['排队人数']<0
df.loc[c1,'排队人数']=0

# df['叫号']=df['小桌叫号']+df['中桌叫号']+df['大桌叫号']
# c12=df['叫号']==0
# df.loc[c12,'排队人数']=0

c2=df['国家']=='CN'
df=df[c2]

c3=df['dt'].isin(['2021-08-13','2022-12-27'])
df=df[~c3]

c4=df['排队时间'].isin('11:30,12:30,13:30,17:30,18:30,19:30,20:30,21:30,22:30,23:30'.split(','))
df=df[c4]

c3=df['dt'].str.contains('2022')
store_2022=df[c3][['门店Id','城市','门店名称']].drop_duplicates(subset='门店Id')

c4=df['dt'].str.contains('2021')
store_2021=df[c4][['门店Id','城市','门店名称']].drop_duplicates(subset='门店Id')

same_store=store_2021.merge(store_2022,on=['门店Id'],how='inner')
df=df[df['门店Id'].isin(same_store['门店Id'])]
Ncount=df.groupby(by=['门店Id'])['排队人数'].count().reset_index()

def workday(s):
    if s in ['Saturday','Sunday']:
        return 'Weekend'
    else:
        return 'Workday'
df['workday']=df['WeekDay'].apply(workday)


print(Ncount['排队人数'].max(),Ncount['排队人数'].min())

102 72


In [4]:
#set 2021 baseline
#用weekday hour 平均，加上周末hourly 平均

c5=df['dt'].str.contains('2021')
df_21=df[c5]
df_21=df_21.groupby(by=['workday','排队时间','城市','门店Id'])['排队人数'].mean().reset_index()
df_21=df_21.groupby(by=['城市','workday'])['排队人数'].sum().reset_index()

d_yichang=pd.pivot_table(df_21,index='城市',columns='workday',values='排队人数',aggfunc='sum').reset_index()
d_yichang['weekendratio']=d_yichang['Weekend']/d_yichang['Workday']
city_delete=d_yichang[d_yichang['weekendratio']<3]['城市']

c6=df_21['城市'].isin(city_delete)
df_21=df_21[~c6]
df_21=df_21.rename(columns={'排队人数':'排队人数21'})
df_21.tail(2)

,城市,workday,排队人数21
472,龙岩,Weekend,2240.0
473,龙岩,Workday,138.0


In [5]:
c7=df['dt'].isin(['2022-12-31','2023-01-01','2023-01-02'])
c8=~df['城市'].isin(city_delete)
df_22=df[c7&c8]
df_22=df_22.groupby(by=['workday','排队时间','城市','门店Id'])['排队人数'].mean().reset_index()
df_22=df_22.groupby(by=['城市','workday'])['排队人数'].sum().reset_index()
df_22.tail(2)

,城市,workday,排队人数
448,龙岩,Weekend,839.0
449,龙岩,Workday,150.0


In [7]:
#make comparable data

hdl=df_22.merge(df_21,on=['城市','workday'],how='left')
hdl['排队_asof21']=hdl['排队人数']/hdl['排队人数21']

hdl=hdl[hdl['workday']=='Weekend']

hdl=hdl.sort_values(by='排队人数21',ascending=False)

c1=hdl['排队_asof21']<2
c2=hdl['排队_asof21']>0.1
hdl=hdl[c1&c2]
d=hdl.copy()
d=d[['城市','排队_asof21']].reset_index(drop=True).rename(columns={'排队_asof21':'hdl'})
d['date']='2023-01-02'
d.to_hdf('Processed_Data/city_compare.h5',key='hdl')

hdl=hdl.head(70).reset_index()
hdl


,index,城市,workday,排队人数,排队人数21,排队_asof21
0,210,杭州,Weekend,137249.0,113182.0,1.212640
1,44,北京,Weekend,137730.0,110255.0,1.249195
2,270,深圳,Weekend,93405.0,84712.0,1.102618
3,2,上海,Weekend,66117.0,80575.0,0.820565
4,152,广州,Weekend,82571.0,70305.0,1.174468
...,...,...,...,...,...,...
65,282,湛江,Weekend,2920.0,5365.0,0.544268
66,326,聊城,Weekend,6213.0,5351.0,1.161091
67,262,淄博,Weekend,3583.0,5079.0,0.705454
68,242,沧州,Weekend,5871.0,5010.0,1.171856


In [11]:
# covid rank vs. HDL vs.2021

%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi

df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')

o=hdl.copy()
o=o.merge(df_covid,on='城市',how='inner')

o=o[['城市', 'progress_2','排队_asof21']]

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)
o.to_clipboard(index=False,header=None)
print('vs. 2021')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
vs. 2021


In [15]:
# 收入 vs. HDL vs.2021

%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')

o=hdl.merge(city_popu,on='城市',how='inner')

o=o[['城市', '人均可支配收入','排队_asof21']]

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)
o.to_clipboard(index=False,header=None)
print('vs. 2021')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
vs. 2021


In [16]:
# d1=pd.read_clipboard()
d2=pd.pivot_table(df[df['dt'].astype('str')>'2022-12-28'],index=['城市'],columns=['dt','排队时间'],values='排队人数',aggfunc='sum').reset_index()
d2=d2.set_index('城市')
d2.reindex(d1['城市'])#.to_clipboard()

NameError: name 'd1' is not defined